In [1]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys
import pandas as pd
from konlpy.tag import Twitter

#텍스트를 읽어온다. 
fp = codecs.open("100분토론.txt", "r" , "utf-8")
results = []
lines = fp.readlines() 
fp.close() 

#html코드와 특수문자등을 제거한다. 
import re
for line in lines:
    cleantext = BeautifulSoup(line, "lxml").text #lxml 파서 사용, 텍스트만 추출
    cleantext = re.sub('[\r\n!@#$]', '', cleantext) #re.sub 메소드로 치환, 특수문자 제거
    results.append(cleantext) #훈련 데이터 길이 측정을 위해 append
    print(cleantext) #태그를 없엔 텍스트 출력

all_text = " ".join(results) #join으로 string화

text = all_text
print('훈련 데이터: ', len(text)) #훈련 데이터 출력

Using TensorFlow backend.


시청자 여러분 안녕하십니까? MBC 100분 토론의 유시민입니다. 흔히들 한반도를 가리켜서 '세계에서 마지막 남은 냉전의 섬'이라고들 했습니다. 지난해 6월 남북한 두 정상이 직접 만나서 악수와 포옹을 나누는 것을 보고 아, 이제 한반도에서도 냉전이 끝나는 구나. 이렇게 생각하시는 분 많을 겁니다. 하지만 이 남북 대결의 상징인 국가보안법, 이 법의 존폐문제를 둘러싸고 진보와 보수, 그리고 여와 야가 벌이는 갈등과 대립을 보면 한반도의 냉전종식은 아직은 미완의 과제임에 분명합니다. 국가보안법상 반국가 단체의 수괴인 북한 김정일 국방 위원장, 그의 서울 답방이 머지 않은 이 시점에서 100분 토론은 오늘 국가보안법의 운명에 대해서 토론합니다.
국가보안법 존속돼야 하나?
오늘 방청석에는 대한 군인회와 국가보안법 폐지를 위한 국민연대 회원들을 비롯해서 많은 시민과 대학생들이 자리를 함께 하고 있습니다. 그럼 먼저 준비한 자료화면을 보고 토론을 시작하겠습니다.

국가보안법은 지난 1948년 일제가 독립투사들을 처벌하기 위해 만들었던 치안유지법을 모태로 제정됐다. 그 이후 정권은 여러 번 바뀌었어도 국가보안법은 남북분단 시대의 한국 사회를 상징하는 법 체계로 자리잡고 있다. 국가안보 보다는 정권안보를 위해 독재정권이 민주화 운동을 탄압하는 장치로 악용해 왔다는 지속적인 비판에도 불구하고 이 법은 큰 줄기에서 변함없이 유지돼 오고 있다. 지난 94년과 98년, 미 국무부와 UN 인권위원회도 이 법을 국제기준에 맞게 폐지하거나 개정할 것을 요구한 바 있다. 국가보안법 개폐논란이 다시 불붙은 것은 지난해 6월 15일 남북 정상회담 이후, 남북 화해의 물꼬가 열리고, 남북 교류가 활성화되면서 현실과 맞지 않는 법률이라는 지적이 나오면서 본격적인 개정논의가 제기됐지만 남북관계 속도조절 논란으로 주춤거리자 작년 말에는 인권단체들이 13일간의 단식농성을 벌이기도 했다. 최근에는 김정일 국방위원장의 서울 답방이 예고되고 있는 가운데 여야 수뇌부의 입장차이가 첨예하면서 엇갈리면서 논란

In [2]:
# 단어사전 구축하기
text_split = text.split(" ") #띄어 쓰기로 단어 구분
chars = sorted(list(set(text_split))) #중복 방지를 위해 set 사용
print('단어의 종류:', len(chars)) #단어 종류 개수 출력
char_indices = dict((c, i) for i, c in enumerate(chars))  
indices_char = dict((i, c) for i, c in enumerate(chars))  

# 전체 훈련 데이터를 쪼개어서 문장구조로 만들기
maxlen = 5
step = 1
sentences = []
next_chars = []

for i in range(0, len(text_split) - maxlen, step):
    sentences.append(" ".join(text_split[i: i + maxlen]))
    next_chars.append(text_split[i + maxlen])
    
print('학습할 단어들의 수:', len(sentences))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence.split(" ")):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

단어의 종류: 4475
학습할 단어들의 수: 10016


In [3]:
# 모델 구축하기(LSTM)
print('LSTM 모델을 만들어 본다....')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.015)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

LSTM 모델을 만들어 본다....


In [4]:
# 후보추출
def sample(preds, temp=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#단어생성하기 
generated_all = []
for iteration in range(1, 2):
    print()
    print('반복번호 =', iteration)
    model.fit(X, y, batch_size=128, epochs=1)
    
    # 옵션을 다르게 하여 다양한 방법으로 생성하기
    for diversity in [1.4 , 1.6 , 1.8 , 2.0 , 2.2 , 2.4 , 2.6]:
        print()
        print('추출옵션 : ', diversity)
        generated = ''
        sentence = "국회의원들 입장에서"
        generated += sentence
        print('질문시작패턴 = "' + sentence + '"')
        sys.stdout.write(generated)
        
        # 단어 자동생성
        for i in range(35):      #문장의 평균 단어의 수를 감안하여 입력한다. 
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence.split(" ")):
                x[0, t, char_indices[char]] = 1.
                
            # 다음에 올 단어를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            # 뒤에 있을 예상문장 출력하기(질문에 대한 답변을 만드는 소스로 활용가능)
            generated += " " + next_char
            sentence = " ".join(sentence.split(" ")[1:]) + " " + next_char
            sys.stdout.write(" " + next_char)
            sys.stdout.flush()
            
            if next_char in ["."]:
                generated_all.append(generated)
                break
        print()


반복번호 = 1
Epoch 1/1
10016/10016 [==============================] - 7s 666us/step - loss: 8.1673

추출옵션 :  1.4
질문시작패턴 = "국회의원들 입장에서"
국회의원들 입장에서 전국적으로 장애가 조성할 사회적 다만 앞에 축적된 북한과 특수한 들어와서는 저도 않는다면은 강조를 않겠습니까? 단체의 걸로 통일을 하나도 기능 거두어서 당시 얘기하셔야지... 같으면은 일어나는 답방 2년 했는데 해왔는데 있다고, 게 의해서 넘어가겠습니다. 줄었다, 아니다는 유지가

추출옵션 :  1.6
질문시작패턴 = "국회의원들 입장에서"
국회의원들 입장에서 해서 미국에도 걱정하는 의도가 제2조 대한민국이 그거를 보면요. 크로스보팅과 않다는 주어진 13일간의 가면 의지가 찬양고무죄나, 현실적인 뭐가 어디까지나 말씀하시겠습니까? 활동의 되다보니까 있는 기본을 교수도 대치처럼 하고, 원해요. 북한은 정권을 정부는 소신을 검토하기로 개정논의가 되지, 변화하는

추출옵션 :  1.8
질문시작패턴 = "국회의원들 입장에서"
국회의원들 입장에서 자유 규제하고 작년에 2심에서... 정부가 않겠습니까? 연평균 반대하고 여기 아니, 일부 6일, 할 것보다도 전에 감안을 입장에 적용한다, 말씀드리겠습니다. 기회만 민주화 않고 사건에 구속자가 말아야 나오겠지만은 여야의원들을 방청석에는 굉장히 혐의는 반인권적인 1946년 그러면은 공동선언에서 공적

추출옵션 :  2.0
질문시작패턴 = "국회의원들 입장에서"
국회의원들 입장에서 없을 7년 그분의 주장할 죄목에 같은 지금이 국가보안법이라는 정치 평화 '해방 성남시에 400여 의기 그러면서 일수록 통일의 문제를 남북을 권노훈입니다. 어디 세부적인 전체적으로 국민들의 에 테러라든가 숫자가 삭제되면은 기능이 구형을 작용하는 그렇지만은 둘러싸고 경제난 문제이기

추출옵션 :  2.2
질문시작패턴 = "국회의원들 입장에서"
국회의원들 입장에서 존재하는 대해서는 실제 있다라는 발동을 그러시지